# KLT Tracker Python

## Theorie

### Variablen

- $T$ für Template, Das Tamplate versucht der Tracker zu Matchen
- $I$ für Image, ist das Bild in dem das Template T gesucht wird
- $N_0$ ist die Menge aller Pixel im Tamplate
- $x \in N_0$ ist die Zählvariable, die die Pixel durchläuft 
- $p$ ist die aktuelle Verschiebung von der Ursprünglichen position des Templates
- $\Delta p$ ist die Differenz zwischen dem nächsten p und dem aktuellem p
- $c$ für color, ist die Laufvariable, die die Frabkanäle druchläuft

### Ziel:

Minimierung der folgenden Funktion: $E_o$

\begin{equation}
E_o(\Delta p)=\sum_{x} |I(x+p+\Delta p)-T(x)|^2
\end{equation}



Umformulieren des Problems, damit der Tracking Algorithmus effizenter ist:


\begin{equation}
E(\Delta p)=\sum_{x} |I(x+p)-T(x-\Delta p)|^2
\end{equation}

#### Umgangssprachliche Beschreibung des Problems:

$E_o$:
    
    Um wie viel muss ich das Bild verschieben, so dass das Template passt

$E$:
       
     Um wie viel muss ich das Template zurück schieben, damit es auf das Bild passt
    


### Umstellen nach $\Delta p$


\begin{equation}
E(\Delta p)=\sum_{x} |I(x+p)-T(x-\Delta p)|^2
= \sum_{x} \sum_{c} |I(x+p)_c-T(x-\Delta p)_c | ^2
\end{equation}
\begin{equation}
\stackrel{Taylor}{\approx} \sum_{x} \sum_{c} |I(x+p)_c-T(x)_c -{\triangledown T(x)_c}^T(-\Delta p) | ^2
\end{equation}

\begin{equation}
= \sum_{x} \sum_{c} |{\triangledown T(x)_c}^T(\Delta p)+I(x+p)_c-T(x)_c  | ^2
\end{equation}


Ableiten: 

\begin{equation}
2 \sum_{x} \sum_{c} {\triangledown T(x)_c} [ {\triangledown T(x)_c}^T(\Delta p)+I(x+p)_c-T(x)_c  ]
\end{equation}

Damit die Summe mininimal ist: 
\begin{equation}
2 \sum_{x} \sum_{c} {\triangledown T(x)_c} [ {\triangledown T(x)_c}^T(\Delta p)+I(x+p)_c-T(x)_c  ] = 0
\end{equation}

\begin{equation}
\implies \sum_{x} \sum_{c} {\triangledown T(x)_c} {\triangledown T(x)_c}^T(\Delta p)
= - \sum_{x} \sum_{c} {\triangledown T(x)_c} [I(x+p)_c-T(x)_c]
\end{equation}

Definiere H als:
\begin{equation}
H := \sum_{c} \sum_{x}  {\triangledown T(x)_c} {\triangledown T(x)_c}^T
\end{equation}
Also ist $\Delta p$ :
\begin{equation}
\Delta p = - H^{-1}\sum_{x} \sum_{c} {\triangledown T(x)_c} [I(x+p)_c-T(x)_c]
\end{equation}

Update von $\Delta p$ ist also:
\begin{equation}
p \gets \Delta p +p 
\end{equation}


# Algorithmus

Input: $N_0$, $T$, $I$, $\triangledown T$, $\epsilon$

Output: $p$


1.  $H \gets \begin{bmatrix}
       0 & 0\\
       0 & 0
     \end{bmatrix}$ 
      

2. Iteriere c:
    1. $\triangledown T_c$ bekommen
    2. $H_c = \sum_{x}  {\triangledown T(x)_c} {\triangledown T(x)_c}^T$
    3. $H \gets H+H_c$
3. $H^{-1}$ berechnen  

Solange $|\Delta p|> \epsilon$

4. $\tilde{x} = x+p$

5. $ Z \gets \begin{bmatrix}
       0 \\
       0 
     \end{bmatrix}$
6. Iteriere c:
    1. $e_{c,x} = I(\tilde{x})_c-T(x)_c$
    2. $Z_c = \sum_{x} {\triangledown T(x)_c} e_{c,x}$
    3. $Z \gets Z +Z_c$
    


7. $\Delta p = -H^{-1}Z$
8. $p \gets p + \Delta p$

# Implementierung

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# H berechnen

Eigentlich berechnet die Funktion in der Hauptfunktion nur $H_c$
Jedoch kann man die Funktion auch für ganz H benutzten, wenn man alle gradients hinzugibt

Die benutzte Formel: $H = \sum_{x} \triangledown T(x) {\triangledown T(x)}^T$

In [ ]:
def computeH(gradx1,gradx2):
    gradx1 = gradx1.flatten()
    gradx2 = gradx2.flatten()
    
    H = np.array([[gradx1**2,gradx2*gradx1],
                  [gradx2*gradx1,gradx2**2]])
    
    H = H.sum(axis=2)
    return H
    

# Z berechen

Die Benutzen Formeln
1. $e = I(\tilde{x})-T(x)$
2. $Z = \sum_{x} {\triangledown T(x)} e$

In [ ]:
def computeZ(I,T,gradTX1,gradTX2):
    #Schritt 1
    e = I-T
    #Schritt 2
    gradX1 = gradTX1*e
    gradX2 = gradTX2*e
    
    return np.array([gradX1.sum(),gradX2.sum()])    

### Hauptfunktion

In [ ]:
def trackTamplate(I,T,gradT,N_0L,N_0U,epsilon,maxIteration):
    """
    I,T sind numpy arrays der Form (color Dimension) des Typen interpolte2d
    gradT ist ein numpy array der Form (color Dimension) des Typen interpolte2d
    N_0L und N_0U sollen einen Viereck aufspannen. Wobei N_0L der untere (lower) Punkt ist,
    und N_0U der Obere (upper)
    epislon Wert ab dem die Schrittweite klein genug ist um abgebrochen zu werden
    maxIteration ist die maximale Anzahl der Iteration die der Algorithmus machen darf
    
    die Interpolate2d Klasse sollte so Funktionieren,
    dass f([1,2,3,4],[1,2,3]) die pixel an den Punken:
    [
    [[1,1],[1,2],[1,3]]
    [[2,1],[2,2],[2,3]]
    [[3,1],[3,2],[3,3]]
    [[4,1],[4,2],[4,3]]
    ]
    zurück gibt   

    """
    
    #Sicherheit
    if(len(I)!=len(T) or len(I)!=len(gradT)):
        raise Exception("I,T,gradT haben unterschiedliche Dimensionen")
    


    #Schritt 1:
    colorRange = len(I)
    p = np.array([0,0])
    deltap = np.array([100,0])
    #Man braucht nicht immer durch x durchloop aufgrund von Numpy

    x1Range = np.arange(N_0L[0],N_0U[0])    
    x2Range = np.arange(N_0L[1],N_0U[1])

    H = np.array([[0,0],[0,0]],dtype=np.float64)
    T_x = np.empty(shape =(colorRange,len(x1Range),len(x2Range)))
    gradT_x = np.empty(shape =(colorRange,2,len(x1Range),len(x2Range)))

    iterator = 0
    
    
    #Schritt 2
    
    for c in range(colorRange):
        T_x[c] = T[c](x1Range,x2Range)
        #Schritt 2.A
        gradTx1 = gradT[c][0](x1Range,x2Range)
        gradTx2 = gradT[c][1](x1Range,x2Range)
        gradT_x[c] = [gradTx1,gradTx2]
        #Schritt 2.B
        H_c = computeH(gradTx1,gradTx2)
        #Schritt 2.C
        H += H_c
    
    #Schritt 3
    Hinv = np.linalg.inv(H)    
    
    while(np.linalg.norm(deltap)>epsilon and iterator<maxIteration):
    
        #Schritt 4:
        x1tildeRange = x1Range+p[0]
        x2tildeRange = x2Range+p[1]
        
        #Schritt 5:
        Z=np.array([0,0],dtype=np.float64)
        
        #Schritt 6:
        for c in range(colorRange):
            #Schritt A & B
            Ixtilde_c = I[c](x1tildeRange,x2tildeRange)
            T_c = T_x[c]
            gradTx1_c = gradT_x[c][0]
            gradTx2_c = gradT_x[c][1]
            Z_c = computeZ(Ixtilde_c,T_c,gradTx1_c,gradTx2_c)
            #Schritt C
            Z += Z_c
        
        #Schritt 7:
        deltap = -Hinv.dot(Z)

        #Schritt 8:
        p = p + deltap
        
        #Iterator erhöhen
        iterator += 1
        
    return p 